In [25]:
import tensorflow as tf
import numpy as np
import pickle
import json
import nltk
import random
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
NEXT_QNT = NEURONS = EPOCHS = 0
with open("../Data/iniV2.pickle", "rb") as f:
    NEXT_QNT , NEURONS, EPOCHS = pickle.load(f)
print(NEURONS,EPOCHS)

64 100


In [3]:
words = []
labels = []
docs_x = []
docs_y = []

with open("../Data/intents.json","r",encoding='utf-8') as file:
    data = json.load(file)

for i,intent in enumerate(data["intents"]):
    intent["tag"] = "tag"+str(i)

with open("../Data/intents.json","w",encoding='utf-8') as file:
    json.dump(data,file,indent=3,ensure_ascii=False)

In [4]:
stopwords = nltk.corpus.stopwords.words('portuguese')
len(stopwords)

204

In [5]:
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])

    if intent["tag"] not in labels:
        labels.append(intent["tag"])

ponctuations = ["?","'",'"',"!",".",","]

#words = [w for w in words if w.lower() not in stopwords]
words = [stemmer.stem(w.lower()) for w in words if w not in ponctuations]
words = sorted(list(set(words)))
labels = sorted(labels)
print(len(words))

561


In [6]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)


training = np.array(training)
output = np.array(output)
N_CLASSES = len(output[0])
SHAPE = len(training[0])

In [7]:
def dense_layers(inputs):
    x = tf.keras.layers.Dense(32,activation='relu')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    
    x = tf.keras.layers.Dense(64,activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    
    x = tf.keras.layers.Dense(128,activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    
    x = tf.keras.layers.Dense(256,activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    return x

def classfier_layer(x,N_CLASSES):
    x = tf.keras.layers.Dense(N_CLASSES,activation='softmax',name='classification')(x)
    return x

def final_model(inputs,N_CLASSES):
    dense = dense_layers(inputs)
    
    classfier = classfier_layer(dense,N_CLASSES)
    
    model = tf.keras.Model(inputs=inputs,outputs=classfier)
    
    return model
    
def define_and_compile_model(SHAPE,N_CLASSES):
    inputs = tf.keras.layers.Input(shape=(SHAPE,))
    
    # create the model
    model = final_model(inputs,N_CLASSES)
    
    # compile your model
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics = {'classification' : 'accuracy'})

    return model

In [10]:
model = define_and_compile_model(SHAPE,N_CLASSES)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 561)]             0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                17984     
_________________________________________________________________
batch_normalization_4 (Batch (None, 32)                128       
_________________________________________________________________
dense_5 (Dense)              (None, 64)                2112      
_________________________________________________________________
batch_normalization_5 (Batch (None, 64)                256       
_________________________________________________________________
dense_6 (Dense)              (None, 128)               8320      
_________________________________________________________________
batch_normalization_6 (Batch (None, 128)               512 

In [11]:
model.fit(training,output,epochs=50)

Epoch 1/50
18/18 [==============================] - 1s 4ms/step - loss: 0.7931 - accuracy: 0.0166
Epoch 2/50
18/18 [==============================] - 0s 5ms/step - loss: 0.7093 - accuracy: 0.0277
Epoch 3/50
18/18 [==============================] - 0s 4ms/step - loss: 0.6415 - accuracy: 0.0832
Epoch 4/50
18/18 [==============================] - 0s 4ms/step - loss: 0.5274 - accuracy: 0.1189
Epoch 5/50
18/18 [==============================] - 0s 3ms/step - loss: 0.3632 - accuracy: 0.1564
Epoch 6/50
18/18 [==============================] - 0s 4ms/step - loss: 0.2119 - accuracy: 0.2172
Epoch 7/50
18/18 [==============================] - 0s 4ms/step - loss: 0.1207 - accuracy: 0.2348
Epoch 8/50
18/18 [==============================] - 0s 4ms/step - loss: 0.0767 - accuracy: 0.2954
Epoch 9/50
18/18 [==============================] - 0s 4ms/step - loss: 0.0561 - accuracy: 0.3983
Epoch 10/50
18/18 [==============================] - 0s 4ms/step - loss: 0.0456 - accuracy: 0.4149
Epoch 11/50
18/18 [

In [12]:
def bag_of_words(s):
    bag = [0 for _ in range(len(words))]

    ponctuations = ["?","'",'"',"!",".",","]

    s_words = nltk.word_tokenize(s)
    s_words = [w for w in s_words if w.lower() not in stopwords]
    s_words = [stemmer.stem(word.lower()) for word in s_words if word not in ponctuations]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    bag = np.array(bag)
    return bag

In [13]:
while True:
    inp = str(input("Digite: "))
    if inp == "quit":
        break
        
    bag = bag_of_words(inp)
    bag = bag.reshape((1,SHAPE))
    predictions = model.predict(bag)
    results_index = np.argmax(predictions)

    tag = labels[results_index]

    for tg in data['intents']:
        if tg['tag'] == tag:
            responses = tg['responses']
            current_tag = tg

    response = random.choice(responses)
    confidence = predictions[0][results_index]
    print("Model:",response,confidence)

Digite: oi
Model: Oi como vai? 0.98542076
Digite: vou bem e você?
Model: Fico feliz em saber que se sente bem 0.97689444
Digite: tá fazendo oque?
Model: <get_ocupied> 0.9951893
Digite: desligar
Model: <finalizar> 0.9742957
Digite: quit


In [14]:
model.save("model.h5")

In [15]:
with open("../Data/intents.json","w",encoding='utf-8') as file:
    json.dump(data,file,indent=3,ensure_ascii=False)

In [16]:
with open("../Data/dataV2.pickle", "wb") as f:
    pickle.dump((words, labels, training, output), f)    

In [20]:
with open("../Data/iniV2.pickle", "wb") as f:
    pickle.dump((120, NEURONS, EPOCHS), f)